In [1]:
import escher
from escher import Builder
import cobra
from time import sleep

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import cobra
import numpy as np
import seaborn as sns
import itertools
from matplotlib.colors import LogNorm, Normalize
import subprocess
import time
import os

In [2]:
escher.rc['never_ask_before_quit'] = True

In [7]:
RNA_dpath =os.path.join('data', 'RNASEQ') 
os.listdir(RNA_dpath)
pmm_df = pd.read_csv(os.path.join(RNA_dpath, 'MED4_PMM2locus.csv'), names=['gene_id', 'pmm'])
deseq_df = pd.read_csv(os.path.join(RNA_dpath, 'MED4_DE_genes_DESEQ_wKEGG.csv'))


In [8]:
# values from Ofaim at el.

PARAMETER_VALUES = [#"Name",     "Reaction ID",          "Lower bound", "UpperBound"
                   ["HCO3",      "HCO3EXcar",            -8,            0],
                   #["Nitrogen",  "AmmoniaEX",            -0.56,         0],
                   #["Phosphate", "FAKEOrthophosphateEX", -0.1,          0],
                   ["Light",     "LightEX",              -150,          0]]
CO2MAX = 0.82

# Block fake reactions
FAKE_TRANSPORT = ["AminosugarsOUT", "FAKEAAOUT", "FAKEABPOUT", "FAKEacpTRANS", "FAKEApoacpTRANS", "FAKEThioredoxinTRANS", 
                  "FreefattyacidsOUT", "7NMeth7carbOUT", "ArtificialproteinOUT", "FADOUT", "LipoylproteinTRANS", "MenaquinoneOUT", 
                  "NicotinateOUT", "THFpolyglutOUT", "Thiamin_dpOUT"]


In [9]:
import os
model_dpath = os.path.join('..', 'Model_files')
model_fname = 'iSO595v7.xml'
model_fpath = os.path.join(model_dpath, model_fname)


In [10]:
model = cobra.io.read_sbml_model(model_fpath)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [11]:
# manipulations copied from Ofaim at el.

remove_blocked = False

# Block H2S
model.reactions.H2SEX.lower_bound = 0

# Block fake transports
for rid in FAKE_TRANSPORT:
    model.reactions.get_by_id(rid).bounds = (0,0)

# Remove blocked reactions
if remove_blocked:
    blocked = cobra.flux_analysis.find_blocked_reactions(model, open_exchanges = True)
    print('blocked', len(blocked), blocked)
    model.remove_reactions([model.reactions.get_by_id(r_id) for r_id in blocked])

# Block maximum CO2 production
model.reactions.CO2EX.bounds = (0, CO2MAX)

for i, row in enumerate(PARAMETER_VALUES):
    # Row: Name, Reaction ID, lower bound, upper bound
    key = row[0]
    reaction_id = row[1]
    lower_bound = row[2]
    upper_bound = row[3]
    r = model.reactions.get_by_id(reaction_id)
    # Fix flux
    r.bounds = (lower_bound, upper_bound)

In [18]:
builder = Builder(model=model)

In [20]:
solution = model.optimize()
builder.reaction_data = solution.fluxes
builder.metabolite_data = solution.shadow_prices

In [21]:
builder.reaction_scale = [
    { 'type': 'min', 'color': '#000000', 'size': 12 },
    { 'type': 'median', 'color': '#ffffff', 'size': 20 },
    { 'type': 'max', 'color': '#ff0000', 'size': 25 }
]

In [22]:
builder.reaction_scale_preset = 'GaBuRd'

In [42]:
builder

Builder(allow_building_duplicate_reactions=False, and_method_in_gene_reaction_rule='mean', cofactors=['atp', '…

In [26]:
r = model.reactions.get_by_id('AmmoniaEX')

In [27]:
dir(r)

['__add__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_annotation',
 '_associate_gene',
 '_check_bounds',
 '_dissociate_gene',
 '_genes',
 '_gpr',
 '_id',
 '_lower_bound',
 '_metabolites',
 '_model',
 '_repr_html_',
 '_set_id_with_model',
 '_update_awareness',
 '_upper_bound',
 'add_metabolites',
 'annotation',
 'boundary',
 'bounds',
 'build_reaction_from_string',
 'build_reaction_string',
 'check_mass_balance',
 'compartments',
 'copy',
 'delete',
 'flux',
 'flux_expression',
 'forward_variable',
 'functional',
 'gene_name

In [28]:
r.products

[]

In [29]:
r.reactants

[<Metabolite Ammonia[e] at 0x7f1d7e6077f0>]

In [30]:
r.metabolites

{<Metabolite Ammonia[e] at 0x7f1d7e6077f0>: -1.0}

In [34]:
ammonia_met = model.metabolites.get_by_id('Ammonia[c]')

In [36]:
dir(ammonia_met)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotation',
 '_bound',
 '_id',
 '_model',
 '_reaction',
 '_repr_html_',
 '_set_id_with_model',
 'annotation',
 'charge',
 'compartment',
 'constraint',
 'copy',
 'elements',
 'formula',
 'formula_weight',
 'id',
 'model',
 'name',
 'notes',
 'reactions',
 'remove_from_model',
 'shadow_price',
 'summary',
 'y']

In [37]:
ammonia_met.reactions

frozenset({<Reaction AmmoniaTRANS at 0x7f1d7e19ed90>,
           <Reaction R00084 at 0x7f1d7e2b9bb0>,
           <Reaction R00131 at 0x7f1d7e3157c0>,
           <Reaction R00189 at 0x7f1d7e327f40>,
           <Reaction R00220 at 0x7f1d7e396610>,
           <Reaction R00253 at 0x7f1d7e33b2b0>,
           <Reaction R00256 at 0x7f1d7e099ca0>,
           <Reaction R00357 at 0x7f1d7e4fcc10>,
           <Reaction R00366 at 0x7f1d7e35bf70>,
           <Reaction R00485 at 0x7f1d7e4fc820>,
           <Reaction R00568 at 0x7f1d7e241ca0>,
           <Reaction R00571 at 0x7f1d7e241fd0>,
           <Reaction R00671 at 0x7f1d7e4b9bb0>,
           <Reaction R00782 at 0x7f1d7e43ff10>,
           <Reaction R00974 at 0x7f1d7e1d7760>,
           <Reaction R00985 at 0x7f1d7e291a30>,
           <Reaction R00996 at 0x7f1d7e3fabe0>,
           <Reaction R00999 at 0x7f1d7e42eb80>,
           <Reaction R01221 at 0x7f1d7e33b730>,
           <Reaction R01230 at 0x7f1d7e219f10>,
           <Reaction R01286 at 0x7

In [41]:
[(r.name, r.reactants) for r in ammonia_met.reactions]

[('L-Asparagine amidohydrolase',
  [<Metabolite ATP[c] at 0x7f1d7e787ee0>,
   <Metabolite Ammonia[c] at 0x7f1d7e787df0>,
   <Metabolite L_Aspartate[c] at 0x7f1d7e787c70>]),
 ('chorismate pyruvate-lyase',
  [<Metabolite Ammonia[c] at 0x7f1d7e787df0>,
   <Metabolite Chorismate[c] at 0x7f1d7e638430>]),
 ('L-Glutamate:ammonia ligase',
  [<Metabolite ATP[c] at 0x7f1d7e787ee0>,
   <Metabolite Ammonia[c] at 0x7f1d7e787df0>,
   <Metabolite L_Glutamate[c] at 0x7f1d7e7877c0>]),
 ('L-cystathionine L-homocysteine-lyase',
  [<Metabolite H2O[c] at 0x7f1d7e787820>,
   <Metabolite L_Cystathionine[c] at 0x7f1d7e5d7430>]),
 ('7-carboxy-7-carbaguanine:ammonia ligase (ADP-forming)',
  [<Metabolite ATP[c] at 0x7f1d7e787ee0>,
   <Metabolite Ammonia[c] at 0x7f1d7e787df0>,
   <Metabolite 7_Carboxy_7_carbaguanine[c] at 0x7f1d7e6071f0>]),
 ('R10949',
  [<Metabolite Ammonia[c] at 0x7f1d7e787df0>,
   <Metabolite Carboxyphosphate[c] at 0x7f1d7e5a5af0>]),
 ('O-Succinyl-L-homoserine succinate-lyase',
  [<Metabolite 

In [ ]:
def build_map(source_reaction, target_reaction):
    pass

def get_child_metabolites(r):
    return r.products

def get_child_reactions(m):
    #[(r.name, r.reactants) for r in ammonia_met.reactions]
    return [r for r in m.reactions if m in r.reactants]
    
    found_reactions = set()
    found_metabolites = set()

    
def _dfs_m(curr_metabolite, target_reaction, found_reactions, found_metabolites):
    if curr_metabolite in found_metabolites:
        return False
    found_metabolites.add(curr_metabolite)
    r_list = get_child_reactions(curr_metabolite)
    for r in r_list:
        
    
    
def _dfs_r(curr_reaction, target_reaction, found_reactions, found_metabolites):
    if curr_reaction == target_reaction:
        return True
    if curr_reaction in found_reactions:
        return False
    found_reactions.add(curr_reaction)
    m_list = get_child_metabolites(curr_reaction)
    
    
    

In [43]:
dir(model)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotation',
 '_compartments',
 '_contexts',
 '_id',
 '_populate_solver',
 '_repr_html_',
 '_sbml',
 '_set_id_with_model',
 '_solver',
 '_tolerance',
 'add_boundary',
 'add_cons_vars',
 'add_groups',
 'add_metabolites',
 'add_reaction',
 'add_reactions',
 'annotation',
 'boundary',
 'compartments',
 'constraints',
 'copy',
 'demands',
 'description',
 'exchanges',
 'genes',
 'get_associated_groups',
 'get_metabolite_compartments',
 'groups',
 'id',
 'medium',
 'merge',
 'metabolites',
 'name',
 'notes',
 'objective',
 'objective_direct

In [46]:
dir(model.groups[0])

['KIND_TYPES',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotation',
 '_id',
 '_kind',
 '_members',
 '_model',
 '_set_id_with_model',
 'add_members',
 'annotation',
 'id',
 'kind',
 'members',
 'name',
 'notes',
 'remove_members']

In [47]:
model.groups[0].name, model.groups[0].members, 

('Alanine, aspartate and glutamate metabolism',
 [<Reaction R00546 at 0x7f1d7e4eabe0>,
  <Reaction R00355 at 0x7f1d7e4fc040>,
  <Reaction R00357 at 0x7f1d7e4fcc10>,
  <Reaction R00369 at 0x7f1d7e4fcf40>,
  <Reaction R00485 at 0x7f1d7e4fc820>,
  <Reaction R00489 at 0x7f1d7e5067c0>,
  <Reaction R00768 at 0x7f1d7e4eaf70>])

In [51]:
model.get_associated_groups(model.reactions.R00369)[0].name

'Alanine, aspartate and glutamate metabolism'

In [53]:
[g.name for g in model.groups]

['Alanine, aspartate and glutamate metabolism',
 'Amino sugar and nucleotide sugar metabolism',
 'Aminoacyl-tRNA biosynthesis',
 'Arginine and proline metabolism',
 'Biomass Formation',
 'Biotin metabolism',
 'Butanoate metabolism',
 'C5-Branched dibasic acid metabolism',
 'Carbon fixation in photosynthetic organisms',
 'Carotenoid biosynthesis',
 'Citrate cycle (TCA cycle)',
 'Cysteine and methionine metabolism',
 'D-Alanine metabolism',
 'D-Glutamine and D-glutamate metabolism',
 'Exchange Reaction',
 'Fake Transport Reaction',
 'Fatty acid biosynthesis',
 'Folate biosynthesis',
 'Fructose and mannose metabolism',
 'Galactose metabolism',
 'Glutathione metabolism',
 'Glycerolipid metabolism',
 'Glycerophospholipid metabolism',
 'Glycine, serine and threonine metabolism',
 'Glycolysis / Gluconeogenesis',
 'Histidine metabolism',
 'Lipopolysaccharide biosynthesis',
 'Lysine biosynthesis',
 'Lysine degradation',
 'Maintenance Reaction',
 'Nicotinate and nicotinamide metabolism',
 'Nitro